In [1]:
pip install --upgrade google-cloud-aiplatform

     |████████████████████████████████| 3.5 MB 17.6 MB/s eta 0:00:01
     |████████████████████████████████| 333 kB 43.1 MB/s eta 0:00:01
     |████████████████████████████████| 2.5 MB 41.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
PROJECT_ID = 'my-project-gemini-415306' # @param (type: 'string')
LOCATION = 'us-central1' # @param (type: 'string')

# Initialize Vertex AI

import vertexai

vertexai.init(project = PROJECT_ID, location = LOCATION)

2024-02-24 07:18:31.474346: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-24 07:18:32.313581: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-24 07:18:32.313705: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-24 07:18:32.494546: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-24 07:18:32.873327: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-24 07:18:32.879164: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
model = GenerativeModel("gemini-pro")

In [4]:
response = model.generate_content(
    "What's the exchange rate for euros to dollars today?"
)
print(response.text)

I do not have access to real-time information and cannot provide the current exchange rate. For the most up-to-date rates, please check a currency converter or financial news website.


In [5]:
user_prompt = "What's the exchange rate from euros to US dollars today?"

response = model.generate_content("""
Your task is to extract parameters from the user's input and return it as a
structured JSON payload. The user will ask about the exchange rate and which
currency they are converting from and converting to.

User input: {user_prompt}

Please extract the currencies as parameters and put them in a JSON object.
""".format(user_prompt=user_prompt))
print(response.text)

```JSON
{
  "parameters": {
    "fromCurrency" : "euros",
    "toCurrency" : "US dollars"
  }
}
```


In [6]:
import requests
url = "https://api.frankfurter.app/latest"
response = requests.get(url)
response.text

'{"amount":1.0,"base":"EUR","date":"2024-02-23","rates":{"AUD":1.6467,"BGN":1.9558,"BRL":5.3831,"CAD":1.4587,"CHF":0.9522,"CNY":7.7972,"CZK":25.354,"DKK":7.4544,"GBP":0.8534,"HKD":8.476,"HUF":388.47,"IDR":16908,"ILS":3.9322,"INR":89.82,"ISK":148.5,"JPY":162.91,"KRW":1440.95,"MXN":18.5089,"MYR":5.1759,"NOK":11.389,"NZD":1.7445,"PHP":60.562,"PLN":4.3118,"RON":4.9744,"SEK":11.1545,"SGD":1.4553,"THB":39.013,"TRY":33.684,"USD":1.0834,"ZAR":20.867}}'

In [8]:
from vertexai.preview.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

model = GenerativeModel("gemini-pro")

In [9]:
get_exchange_rate_func = FunctionDeclaration(
    name="get_exchange_rate",
    description="Get the exchange rate for currencies between countries",
    parameters={
    "type": "object",
    "properties": {
        "currency_date": {
            "type": "string",
            "description": "A date that must always be in YYYY-MM-DD format or the value 'latest' if a time period is not specified"
        },
        "currency_from": {
            "type": "string",
            "description": "The currency to convert from in ISO 4217 format"
        },
        "currency_to": {
            "type": "string",
            "description": "The currency to convert to in ISO 4217 format"
        }
    },
         "required": [
            "currency_from",
            "currency_date",
      ]
  },
)

In [10]:
exchange_rate_tool = Tool(
    function_declarations=[get_exchange_rate_func],
)

In [11]:
prompt = """What is the exchange rate from Australian dollars to Swedish krona?
How much is 500 Australian dollars worth in Swedish krona?"""

response = model.generate_content(
    prompt,
    tools=[exchange_rate_tool],
)

In [13]:
print(response.candidates[0].content)

role: "model"
parts {
  function_call {
    name: "get_exchange_rate"
    args {
      fields {
        key: "currency_to"
        value {
          string_value: "SEK"
        }
      }
      fields {
        key: "currency_from"
        value {
          string_value: "AUD"
        }
      }
      fields {
        key: "currency_date"
        value {
          string_value: "latest"
        }
      }
    }
  }
}



In [14]:
params = {}
for key, value in response.candidates[0].content.parts[0].function_call.args.items():
    params[key[9:]] = value
params

{'from': 'AUD', 'date': 'latest', 'to': 'SEK'}

In [15]:
import requests
url = f"https://api.frankfurter.app/{params['date']}"
api_response = requests.get(url, params=params)
api_response.text

'{"amount":1.0,"base":"AUD","date":"2024-02-23","rates":{"SEK":6.7739}}'

In [16]:
response = model.generate_content(
    [
    Content(role="user", parts=[
        Part.from_text(prompt + """Give your answer in steps with lots of detail
            and context, including the exchange rate and date."""),
    ]),
    Content(role="function", parts=[
        Part.from_dict({
            "function_call": {
                "name": "get_exchange_rate",
            }
        })
    ]),
    Content(role="function", parts=[
        Part.from_function_response(
            name="get_exchange_rate",
            response={
                "content": api_response.text,
            }
        )
    ]),
    ],
    tools=[exchange_rate_tool],
)


response.candidates[0].content.parts[0].text

'The exchange rate from Australian dollars to Swedish krona on 2024-02-23 is 1 AUD = 6.7739 SEK.\n\nTherefore, 500 Australian dollars is worth 500 * 6.7739 = 3386.95 Swedish krona.'